<a href="https://colab.research.google.com/github/SoniaPMi/AI-3-RL/blob/main/ddpg_mountaincar_cont.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Deterministic Policy Gradient (DDPG)

**Author:** [amifunny](https://github.com/amifunny)<br>
**Date created:** 2020/06/04<br>
**Last modified:** 2020/09/21<br>
**Description:** Implementing DDPG algorithm on the Inverted Pendulum Problem.

## Introduction

**Deep Deterministic Policy Gradient (DDPG)** is a model-free off-policy algorithm for
learning continous actions.

It combines ideas from DPG (Deterministic Policy Gradient) and DQN (Deep Q-Network).
It uses Experience Replay and slow-learning target networks from DQN, and it is based on
DPG,
which can operate over continuous action spaces.

This tutorial closely follow this paper -
[Continuous control with deep reinforcement learning](https://arxiv.org/pdf/1509.02971.pdf)

## Problem

We are trying to solve the classic **Inverted Pendulum** control problem.
In this setting, we can take only two actions: swing left or swing right.

What make this problem challenging for Q-Learning Algorithms is that actions
are **continuous** instead of being **discrete**. That is, instead of using two
discrete actions like `-1` or `+1`, we have to select from infinite actions
ranging from `-2` to `+2`.

## Quick theory

Just like the Actor-Critic method, we have two networks:

1. Actor - It proposes an action given a state.
2. Critic - It predicts if the action is good (positive value) or bad (negative value)
given a state and an action.

DDPG uses two more techniques not present in the original DQN:

**First, it uses two Target networks.**

**Why?** Because it add stability to training. In short, we are learning from estimated
targets and Target networks are updated slowly, hence keeping our estimated targets
stable.

Conceptually, this is like saying, "I have an idea of how to play this well,
I'm going to try it out for a bit until I find something better",
as opposed to saying "I'm going to re-learn how to play this entire game after every
move".
See this [StackOverflow answer](https://stackoverflow.com/a/54238556/13475679).

**Second, it uses Experience Replay.**

We store list of tuples `(state, action, reward, next_state)`, and instead of
learning only from recent experience, we learn from sampling all of our experience
accumulated so far.

Now, let's see how is it implemented.

In [1]:
import gym
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


## MountainCarContinuous v0
Recently, it has been used to compare DDPG to CMA-ES in this paper:

http://arxiv.org/abs/1606.09152

### Description

An underpowered car must climb a one-dimensional hill to reach a target. Unlike MountainCar v0, the action (engine force applied) is allowed to be a continuous value.

The target is on top of a hill on the right-hand side of the car. If the car reaches it or goes beyond, the episode terminates.

On the left-hand side, there is another hill. Climbing this hill can be used to gain potential energy and accelerate towards the target. On top of this second hill, the car cannot go further than a position equal to -1, as if there was a wall. Hitting this limit does not generate a penalty (it might in a more challenging version).


### Observation

Type: Box(2)

| Num  |	Observation |	Min  |	Max |
|------|--------------|------|------|
| 0    | Car Position |	-1.2 | 0.6  |
|------|--------------|------|------|
| 1    | Car Velocity |	-0.07| 	0.07|

Note that velocity has been constrained to facilitate exploration, but this constraint might be relaxed in a more challenging version.

### Reward
Reward is 100 for reaching the target of the hill on the right hand side, minus the squared sum of actions from start to goal.

This reward function raises an exploration challenge, because if the agent does not reach the target soon enough, it will figure out that it is better not to move, and won't find the target anymore.

Note that this reward is unusual with respect to most published work, where the goal was to reach the target as fast as possible, hence favouring a bang-bang strategy.


### Starting State
Position between -0.6 and -0.4, null velocity.

### Episode Termination
Position equal to 0.5. A constraint on velocity might be added in a more challenging version.

Adding a maximum number of steps might be a good idea.

### Solved Requirements
Get a reward over 90. This value might be tuned.

We use [OpenAIGym](http://gym.openai.com/docs) to create the environment.
We will use the `upper_bound` parameter to scale our actions later.

In [2]:
problem = "MountainCarContinuous-v0"
env = gym.make(problem)

num_states = env.observation_space.shape[0]
print("Size of State Space ->  {}".format(num_states))
num_actions = env.action_space.shape[0]
print("Size of Action Space ->  {}".format(num_actions))

upper_bound = env.action_space.high[0]
lower_bound = env.action_space.low[0]

print("Max Value of Action ->  {}".format(upper_bound))
print("Min Value of Action ->  {}".format(lower_bound))

Size of State Space ->  2
Size of Action Space ->  1
Max Value of Action ->  1.0
Min Value of Action ->  -1.0


To implement better exploration by the Actor network, we use noisy perturbations,
specifically
an **Ornstein-Uhlenbeck process** for generating noise, as described in the paper.
It samples noise from a correlated normal distribution.

In [3]:

class OUActionNoise:
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None):
        self.theta = theta
        self.mean = mean
        self.std_dev = std_deviation
        self.dt = dt
        self.x_initial = x_initial
        self.reset()

    def __call__(self):
        # Formula taken from https://www.wikipedia.org/wiki/Ornstein-Uhlenbeck_process.
        x = (
            self.x_prev
            + self.theta * (self.mean - self.x_prev) * self.dt
            + self.std_dev * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        )
        # Store x into x_prev
        # Makes next noise dependent on current one
        self.x_prev = x
        return x

    def reset(self):
        if self.x_initial is not None:
            self.x_prev = self.x_initial
        else:
            self.x_prev = np.zeros_like(self.mean)


The `Buffer` class implements Experience Replay.

---
![Algorithm](https://i.imgur.com/mS6iGyJ.jpg)
---


**Critic loss** - Mean Squared Error of `y - Q(s, a)`
where `y` is the expected return as seen by the Target network,
and `Q(s, a)` is action value predicted by the Critic network. `y` is a moving target
that the critic model tries to achieve; we make this target
stable by updating the Target model slowly.

**Actor loss** - This is computed using the mean of the value given by the Critic network
for the actions taken by the Actor network. We seek to maximize this quantity.

Hence we update the Actor network so that it produces actions that get
the maximum predicted value as seen by the Critic, for a given state.

In [4]:

class Buffer:
    def __init__(self, buffer_capacity=100000, batch_size=64):
        # Number of "experiences" to store at max
        self.buffer_capacity = buffer_capacity
        # Num of tuples to train on.
        self.batch_size = batch_size

        # Its tells us num of times record() was called.
        self.buffer_counter = 0

        # Instead of list of tuples as the exp.replay concept go
        # We use different np.arrays for each tuple element
        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, num_actions))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))

    # Takes (s,a,r,s') obervation tuple as input
    def record(self, obs_tuple):
        # Set index to zero if buffer_capacity is exceeded,
        # replacing old records
        index = self.buffer_counter % self.buffer_capacity

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]

        self.buffer_counter += 1

    # Eager execution is turned on by default in TensorFlow 2. Decorating with tf.function allows
    # TensorFlow to build a static graph out of the logic and computations in our function.
    # This provides a large speed up for blocks of code that contain many small TensorFlow operations such as this one.
    @tf.function
    def update(
        self, state_batch, action_batch, reward_batch, next_state_batch,
    ):
        # Training and updating Actor & Critic networks.
        # See Pseudo Code.
        with tf.GradientTape() as tape:
            target_actions = target_actor(next_state_batch, training=True)
            y = reward_batch + gamma * target_critic(
                [next_state_batch, target_actions], training=True
            )
            critic_value = critic_model([state_batch, action_batch], training=True)
            critic_loss = tf.math.reduce_mean(tf.math.square(y - critic_value))

        critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables)
        critic_optimizer.apply_gradients(
            zip(critic_grad, critic_model.trainable_variables)
        )

        with tf.GradientTape() as tape:
            actions = actor_model(state_batch, training=True)
            critic_value = critic_model([state_batch, actions], training=True)
            # Used `-value` as we want to maximize the value given
            # by the critic for our actions
            actor_loss = -tf.math.reduce_mean(critic_value)

        actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables)
        actor_optimizer.apply_gradients(
            zip(actor_grad, actor_model.trainable_variables)
        )

    # We compute the loss and update parameters
    def learn(self):
        # Get sampling range
        record_range = min(self.buffer_counter, self.buffer_capacity)
        # Randomly sample indices
        batch_indices = np.random.choice(record_range, self.batch_size)

        # Convert to tensors
        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])

        self.update(state_batch, action_batch, reward_batch, next_state_batch)


# This update target parameters slowly
# Based on rate `tau`, which is much less than one.
@tf.function
def update_target(target_weights, weights, tau):
    for (a, b) in zip(target_weights, weights):
        a.assign(b * tau + a * (1 - tau))


Here we define the Actor and Critic networks. These are basic Dense models
with `ReLU` activation.

Note: We need the initialization for last layer of the Actor to be between
`-0.003` and `0.003` as this prevents us from getting `1` or `-1` output values in
the initial stages, which would squash our gradients to zero,
as we use the `tanh` activation.

In [5]:

def get_actor():
    # Initialize weights between -3e-3 and 3-e3
    last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)

    inputs = layers.Input(shape=(num_states,))
    out = layers.Dense(256, activation="relu")(inputs)
    out = layers.Dense(256, activation="relu")(out)
    outputs = layers.Dense(1, activation="tanh", kernel_initializer=last_init)(out)

    # Our upper bound is 2.0 for Pendulum.
    outputs = outputs * upper_bound
    model = tf.keras.Model(inputs, outputs)
    return model


def get_critic():
    # State as input
    state_input = layers.Input(shape=(num_states))
    state_out = layers.Dense(16, activation="relu")(state_input)
    state_out = layers.Dense(32, activation="relu")(state_out)

    # Action as input
    action_input = layers.Input(shape=(num_actions))
    action_out = layers.Dense(32, activation="relu")(action_input)

    # Both are passed through seperate layer before concatenating
    concat = layers.Concatenate()([state_out, action_out])

    out = layers.Dense(256, activation="relu")(concat)
    out = layers.Dense(256, activation="relu")(out)
    outputs = layers.Dense(1)(out)

    # Outputs single value for give state-action
    model = tf.keras.Model([state_input, action_input], outputs)

    return model


`policy()` returns an action sampled from our Actor network plus some noise for
exploration.

In [6]:

def policy(state, noise_object):
    sampled_actions = tf.squeeze(actor_model(state))
    noise = noise_object()
    # Adding noise to action
    sampled_actions = sampled_actions.numpy() + noise

    # We make sure action is within bounds
    legal_action = np.clip(sampled_actions, lower_bound, upper_bound)

    return [np.squeeze(legal_action)]


## Training hyperparameters

In [7]:
std_dev = 0.2
ou_noise = OUActionNoise(mean=np.zeros(1), std_deviation=float(std_dev) * np.ones(1))

actor_model = get_actor()
critic_model = get_critic()

target_actor = get_actor()
target_critic = get_critic()

# Making the weights equal initially
target_actor.set_weights(actor_model.get_weights())
target_critic.set_weights(critic_model.get_weights())

# Learning rate for actor-critic models
critic_lr = 0.002
actor_lr = 0.001

critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

total_episodes = 10000
# Discount factor for future rewards
gamma = 0.99
# Used to update target networks
tau = 0.005

buffer = Buffer(50000, 64)

Now we implement our main training loop, and iterate over episodes.
We sample actions using `policy()` and train with `learn()` at each time step,
along with updating the Target networks at a rate `tau`.

In [ ]:
# To store reward history of each episode
ep_reward_list = []
# To store average reward history of last few episodes
avg_reward_list = []

# Takes about 4 min to train
for ep in range(total_episodes):

    prev_state = env.reset()
    episodic_reward = 0

    while True:
        # Uncomment this to see the Actor in action
        # But not in a python notebook.
        # env.render()

        tf_prev_state = tf.expand_dims(tf.convert_to_tensor(prev_state), 0)

        action = policy(tf_prev_state, ou_noise)
        # Recieve state and reward from environment.
        state, reward, done, info = env.step(action)

        buffer.record((prev_state, action, reward, state))
        episodic_reward += reward

        buffer.learn()
        update_target(target_actor.variables, actor_model.variables, tau)
        update_target(target_critic.variables, critic_model.variables, tau)

        # End this episode when `done` is True
        if done:
            break

        prev_state = state

    ep_reward_list.append(episodic_reward)

    # Mean of last 40 episodes
    avg_reward = np.mean(ep_reward_list[-40:])
    print("Episode * {} * Avg Reward is ==> {}".format(ep, avg_reward))
    avg_reward_list.append(avg_reward)

# Plotting graph
# Episodes versus Avg. Rewards
plt.plot(avg_reward_list)
plt.xlabel("Episode")
plt.ylabel("Avg. Epsiodic Reward")
plt.show()

Episode * 0 * Avg Reward is ==> -1.4043965904086102
Episode * 1 * Avg Reward is ==> -4.607687020181968
Episode * 2 * Avg Reward is ==> -5.60782704617294
Episode * 3 * Avg Reward is ==> -10.617670349527113
Episode * 4 * Avg Reward is ==> -11.50541700644045
Episode * 5 * Avg Reward is ==> -12.332493380963259
Episode * 6 * Avg Reward is ==> -19.76809351938647
Episode * 7 * Avg Reward is ==> -20.40492788570947
Episode * 8 * Avg Reward is ==> -18.879704581932824
Episode * 9 * Avg Reward is ==> -20.14118170964789
Episode * 10 * Avg Reward is ==> -21.585531341794027
Episode * 11 * Avg Reward is ==> -20.157900318384055
Episode * 12 * Avg Reward is ==> -19.439133800217913
Episode * 13 * Avg Reward is ==> -18.485093364328286
Episode * 14 * Avg Reward is ==> -17.720941376129872
Episode * 15 * Avg Reward is ==> -16.774120365705514
Episode * 16 * Avg Reward is ==> -16.50752113267552
Episode * 17 * Avg Reward is ==> -15.756857071902807
Episode * 18 * Avg Reward is ==> -15.252092019300445
Episode * 1

Problem solved at:<br>
Episode * 0 * Avg Reward is ==> -22.5915<br>
Episode * 1 * Avg Reward is ==> -12.2777<br>
Episode * 2 * Avg Reward is ==> -13.2404<br>
Episode * 3 * Avg Reward is ==> -11.6383<br>
...<br>
Episode * 531 * Avg Reward is ==> -32.249<br>
Episode * 532 * Avg Reward is ==> -29.686<br>
Episode * 533 * Avg Reward is ==> -27.247<br>
Episode * 534 * Avg Reward is ==> -24.820<br>
Episode * 535 * Avg Reward is ==> -22.506<br>
Episode * 536 * Avg Reward is ==> -19.842<br>
Episode * 537 * Avg Reward is ==> -17.377<br>
Episode * 538 * Avg Reward is ==> -14.759<br>
Episode * 539 * Avg Reward is ==> -12.135<br>
Episode * 540 * Avg Reward is ==> -9.1966<br>
Episode * 541 * Avg Reward is ==> -6.1309<br>
Episode * 542 * Avg Reward is ==> -5.8332<br>
Episode * 543 * Avg Reward is ==> -1.1396<br>
Episode * 544 * Avg Reward is ==> 3.33284<br>
Episode * 545 * Avg Reward is ==> 6.74444<br>
Episode * 546 * Avg Reward is ==> 7.04198<br>
Episode * 547 * Avg Reward is ==> 11.5895<br>
Episode * 548 * Avg Reward is ==> 15.1776<br>
Episode * 549 * Avg Reward is ==> 19.0443<br>
Episode * 550 * Avg Reward is ==> 23.4529<br>
Episode * 551 * Avg Reward is ==> 26.7377<br>
Episode * 552 * Avg Reward is ==> 30.7801<br>
Episode * 553 * Avg Reward is ==> 34.6756<br>
Episode * 554 * Avg Reward is ==> 38.3789<br>
Episode * 555 * Avg Reward is ==> 42.1994<br>
Episode * 556 * Avg Reward is ==> 46.6344<br>
Episode * 557 * Avg Reward is ==> 51.2720<br>
Episode * 558 * Avg Reward is ==> 55.8356<br>
Episode * 559 * Avg Reward is ==> 60.6907<br>
Episode * 560 * Avg Reward is ==> 65.1503<br>
Episode * 561 * Avg Reward is ==> 69.0114<br>
Episode * 562 * Avg Reward is ==> 73.0471<br>
Episode * 563 * Avg Reward is ==> 77.827<br>
Episode * 564 * Avg Reward is ==> 82.5441<br>
Episode * 565 * Avg Reward is ==> 87.095<br>
Episode * 566 * Avg Reward is ==> 91.909<br>
Episode * 567 * Avg Reward is ==> 92.094<br>
Episode * 568 * Avg Reward is ==> 92.075<br>
Episode * 569 * Avg Reward is ==> 92.081<br>
Episode * 570 * Avg Reward is ==> 92.044<br>
Episode * 571 * Avg Reward is ==> 91.977<br>
...<br>
Episode * 2885 * Avg Reward is ==> 93.780<br>
Episode * 2886 * Avg Reward is ==> 93.777<br>
Episode * 2887 * Avg Reward is ==> nan<br>
Episode * 2888 * Avg Reward is ==> nan<br>

If training proceeds correctly, the average episodic reward will increase with time.

Feel free to try different learning rates, `tau` values, and architectures for the
Actor and Critic networks.

The Inverted Pendulum problem has low complexity, but DDPG work great on many other
problems.

Another great environment to try this on is `LunarLandingContinuous-v2`, but it will take
more episodes to obtain good results.

In [ ]:
# Save the weights
actor_model.save_weights("mcar_cont_actor.h5")
critic_model.save_weights("mcar_cont_critic.h5")

target_actor.save_weights("mcar_cont_target_actor.h5")
target_critic.save_weights("mcar_cont_target_critic.h5")

In [ ]:
def no_noise():
  return 0


prev_state = env.reset()

trajectory = [prev_state]
episodic_reward = 0
steps = 0
while True:
    # Uncomment this to see the Actor in action
    # But not in a python notebook.
    # env.render()

    tf_prev_state = tf.expand_dims(tf.convert_to_tensor(prev_state), 0)

    action = policy(tf_prev_state, no_noise)
    # Recieve state and reward from environment.
    state, reward, done, info = env.step(action)
    trajectory.append(state)
    episodic_reward += reward

    # End this episode when `done` is True
    if done:
        break
    # print(steps, state.round(2), action[0].round(3), round(reward,3))
    prev_state = state
    steps += 1

In [ ]:
# Num 	Observation 	Min 	Max
# 0 	Car Position 	-1.2 	0.6
# 1 	Car Velocity 	-0.07 	0.07

x_min, x_max = -1.2, 0.6
y_min, y_max = -0.07, 0.07

x = np.linspace(x_min, x_max, 100, endpoint=True)
y = np.linspace(y_min, y_max, 100, endpoint=True)
xx, yy = np.meshgrid(x, y, sparse=False, indexing='xy')
grid_pairs = np.hstack([xx.reshape(-1, 1), yy.reshape(-1, 1)])

tf_states = tf.convert_to_tensor(grid_pairs)

actions = policy(tf_states, no_noise)[0]
ff = np.reshape(actions, xx.shape)


fig = plt.figure(figsize=(12, 12))
ax = fig.gca()
ax.contourf(x, y, ff)
ax.set_xlim([x_min, x_max])
ax.set_ylim([y_min, y_max])
ax.set_xlim([x_min, x_max])
ax.set_ylim([y_min, y_max])
ax.plot(trajectory_mat[:,0], trajectory_mat[:,1], 'k')
plt.show()

fig = plt.figure(figsize=(12, 12))
ax = fig.gca()
cfset = ax.contourf(xx, yy, ff, cmap='coolwarm')
cset = ax.contour(xx, yy, ff, colors='k')
ax.clabel(cset, inline=1, fontsize=10)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.plot(trajectory_mat[:,0], trajectory_mat[:,1], 'k')
plt.title('2D Gaussian Kernel density estimation')